low: 1.0 from 0% up to 20%, linear decline up to 40%  
moderate: linear increase from 30% to 40%, 1.0 from 40% to 60%, decline from 60% to 70%  
high:  increase from 0 at 60% to 80%, plateau at 80% to 100%

Compare rise and fall using minimum instead of switchcase/nested if/else.
NumPy operations like (x - a)/(b - a), np.minimum, and np.clip are implemented in compiled C under the hood.
They operate on the entire array at once without Python loops or conditionals.

In [102]:
# --------- Helper function for trapezoidal membership ---------
def trapezoid(x, a, b, c, d):
    """
    Vectorized trapezoidal membership function.
    x: scalar or numpy array
    a, b: start and top-left of trapezoid
    c, d: top-right and end of trapezoid
    Returns same shape as x.
    """
    x = np.asarray(x, dtype=float)

    # Rising edge
    rise = np.zeros_like(x)
    if b != a:
        rise = (x - a) / (b - a)
    else:
        rise[x >= b] = 1.0  # vertical rise, there is no rise

    # Falling edge
    fall = np.zeros_like(x)
    if d != c:
        fall = (d - x) / (d - c)
    else: 
        fall[x <= c] = 1.0  # vertical fall, there is no fall

    # Compute membership: min of rising, plateau=1, falling. 
    # Elements outisde rise will be 1.0, which means it will choose fall value if less than 1, or keep 1 if plateau.
    membership = np.minimum(np.minimum(rise, 1.0), fall)

    # Clip to [0,1]
    return np.clip(membership, 0.0, 1.0)


In [103]:
import geopandas as gpd
from rasterio.transform import from_origin
import rasterio
from rasterio import features
import numpy as np

def generate_aoi_mask(boundary_path):
    # Read boundary shapefile
    boundary = gpd.read_file(boundary_path)
    minx, miny, maxx, maxy = boundary.total_bounds

    # Compute rows/columns
    pixel_size=30
    width  = int(np.ceil((maxx - minx) / pixel_size))
    height = int(np.ceil((maxy - miny) / pixel_size))

    # Build transform (top-left origin)
    transform = from_origin(minx, maxy, pixel_size, pixel_size)

    # Raster profile for all your layers
    profile = {
        "driver": "GTiff",
        "dtype": "float32",
        "nodata": np.nan,
        "width": width,
        "height": height,
        "count": 1,
        "crs": boundary.crs,
        "transform": transform,
    }

    # ➤ Rasterize the boundary polygon (mask)
    aoi_mask = features.rasterize(
        [(geom, 1) for geom in boundary.geometry],
        out_shape=(height, width),
        transform=transform,
        fill=0,
        all_touched=True,
        dtype="float32"
    )

    # Save AOI mask grid
    out_path = r"Base_Layers\Upper_Marikina_Watershed_rasterized.tif"
    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(aoi_mask, 1)
    
    print(f"Aoi grid raster saved: {out_path}")
    print(f"Grid created: {width} x {height} pixels at {pixel_size}m resolution")

    return aoi_mask, profile

In [2]:
import geopandas as gpd
from rasterio.transform import from_origin
import rasterio
import numpy as np
from rasterio.windows import from_bounds
import os

def clip_component_layers(mask_path,layer_path):

    with rasterio.open(mask_path) as mask:
        mask_data = mask.read(1)
        mask_transform = mask.transform
        mask_crs = mask.crs
        mask_height = mask.height
        mask_width = mask.width
        mask_bounds = mask.bounds
        mask_profile = mask.profile.copy()
        pixel_size_x = mask_transform.a

    with rasterio.open(layer_path) as layer:
        layer_data = layer.read()
        layer_transform = layer.transform
        layer_crs = layer.crs
        layer_height = layer.height
        layer_width = layer.width
        layer_profile = layer.profile.copy()
        layer_size_x = layer_transform.a

        if layer.crs != mask_crs:
            raise ValueError("Mask and layer CRS do not match")

        # Create window from mask bounds
        window = from_bounds(
            *mask_bounds,
            transform=layer.transform
        )

        layer_data = layer.read(1, window=window)
        layer_transform = layer.window_transform(window)

    # Apply mask (assuming mask == 0 is outside)
    masked_layer = np.where(mask_data==0, mask_profile.get("nodata", -9999), layer_data)

    # Update output profile
    mask_profile.update({
        "height": masked_layer.shape[0],
        "width": masked_layer.shape[1],
        "transform": layer_transform,
        "count": 1
    })


    # Write output
    output_path = r"Component_Layers\aspect_-9999flat_Marikina_WS.tif"
    with rasterio.open(output_path, "w", **mask_profile) as dst:
        dst.write(masked_layer,1)

    # Print output properties
    pixel_size_x = layer_transform.a
    pixel_size_y = -layer_transform.e

    print("Clipping completed")
    print(f"Output height: {masked_layer.shape[0]}")
    print(f"Output width:  {masked_layer.shape[1]}")
    print(f"Resolution:   {pixel_size_x} x {pixel_size_y}")


mask_path = r"Base_Layers\Upper_Marikina_Watershed_rasterized.tif"
layer_path = r"D:\Ralph\research\common_maps\aspect_from_DEM_ifsar_30m.tif"
result = clip_component_layers(mask_path,layer_path)

#AT_86_05_PH_2023

Clipping completed
Output height: 976
Output width:  496
Resolution:   30.0 x 29.999999999999996


In [ ]:
# --------- Annual Rainfall Layer ------------
# --------- 2-2-1_PH_AR-ANN-RF-1986-2005_MO_2023
# --------- Read raster ---------
def fuzzy_AR():
    raster_path = rf"Component_Layers\AR-86-05-Marikina_WS.tif"
    layer = "AR"
    print(f"Fuzzy rasters for {layer} starting")

    with rasterio.open(raster_path) as src:
        raster = src.read(1).astype(float)  # Read first band
        profile = src.profile

    # --------- Normalize raster to 0-1 ---------
    nodata = -9999
    raster = np.where(raster == nodata, np.nan, raster)

    raster_min = np.nanmin(raster)
    raster_max = np.nanmax(raster)
    print(f"raster_min = {raster_min}")
    print(f"raster_max = {raster_max}")

    # --------- Normalize raster ---------
    denom = raster_max - raster_min
    if denom == 0:
        print("WARNING: raster_min == raster_max — normalization forced to 0")
        raster_norm = np.zeros_like(raster, dtype=float)
    else:
        raster_norm = (raster - raster_min) / denom
    
    # --------- Compute fuzzy memberships ---------
    # To avoid zero membership for fuzzy AND, make functions reach 0 and 1
    low_AR = trapezoid(raster_norm, 0.0, 0.0, 0.2, 1)
    moderate_AR = trapezoid(raster_norm, 0, 0.4, 0.6, 1)
    high_AR = trapezoid(raster_norm, 0, 0.8, 1.0, 1.0)


    # --------- Save fuzzy rasters ---------
    for layer, name, data in zip([layer]*3, ["low", "moderate", "high"], [low_AR, moderate_AR, high_AR]):
        out_path = rf"Fuzzy_Layers\{layer}_fuzzy_{name}.tif"
        profile.update(dtype=rasterio.float32, count=1)
        with rasterio.open(out_path, "w", **profile) as dst:
            dst.write(data.astype(rasterio.float32), 1)
        print(f"Fuzzy raster saved: {out_path}")

    print(f"Fuzzy rasters for {layer} completed")
    return low_AR, moderate_AR, high_AR

In [4]:
# --------- Elevation Layer ------------
# --------- COG_SRTM90m_CGIAR-CSI_2018 = elevation-upper-marikina-watershed-SRTM.
# --------- DEM_ifsar_30m_phil.tif = NAMRIA 2013
# --------- Read raster ---------
def fuzzy_Elev():
    raster_path = rf"Component_Layers\elevation_upper_marikina_watershed_DEM.tif"
    layer = "elev"
    print(f"Fuzzy rasters for {layer} starting")
    
    with rasterio.open(raster_path) as src:
        raster = src.read(1).astype(float)  # Read first band
        profile = src.profile

    # --------- Normalize raster to 0-1 ---------
    nodata = -9999
    raster = np.where(raster == nodata, np.nan, raster)

    raster_min = np.nanmin(raster)
    raster_max = np.nanmax(raster)
    print(f"raster_min = {raster_min}")
    print(f"raster_max = {raster_max}")

    # --------- Normalize raster ---------
    denom = raster_max - raster_min
    if denom == 0:
        print("WARNING: raster_min == raster_max — normalization forced to 0")
        raster_norm = np.zeros_like(raster, dtype=float)
    else:
        raster_norm = (raster - raster_min) / denom

    # Replace nan with a placeholder (optional)
    #raster_to_save = np.where(np.isnan(raster_norm), -9999, raster_norm)

    # Save as text
    #np.savetxt("normalized_raster.txt", raster_to_save, fmt="%.4f")
    
    # --------- Compute fuzzy memberships ---------
    # To avoid zero membership for fuzzy AND, make functions reach 0 and 1
    low_elev = trapezoid(raster_norm, 0.0, 0.0, 0.2, 1)
    moderate_elev = trapezoid(raster_norm, 0, 0.4, 0.6, 1)
    high_elev = trapezoid(raster_norm, 0, 0.8, 1.0, 1.0)


    # --------- Save fuzzy rasters ---------
    for layer, name, data in zip([layer]*3, ["low", "moderate", "high"], [low_elev, moderate_elev, high_elev]):
        out_path = rf"Fuzzy_Layers\{layer}_fuzzy_{name}.tif"
        profile.update(dtype=rasterio.float32, count=1)
        with rasterio.open(out_path, "w", **profile) as dst:
            dst.write(data.astype(rasterio.float32), 1)
        print(f"Fuzzy raster saved: {out_path}")

    print(f"Fuzzy rasters for {layer} completed")
    return low_elev, moderate_elev, high_elev, profile

In [ ]:
# --------- River Proximity Layer ------------
# --------- 1-3-5_PH_River_MO-GED_2020.shp.
import rasterio
from rasterio import features
import numpy as np
import geopandas as gpd
from scipy.ndimage import distance_transform_edt

def river_distance_raster():
    print("Computing river distance raster (aligned to AOI grid)…")

    # Load river shapefile

    watershed_raster_path = r"Base_Layers\Upper_Marikina_Watershed_rasterized.tif"

    with rasterio.open(watershed_raster_path) as ws:
        ws_data = ws.read(1)
        ws_transform = ws.transform
        ws_crs = ws.crs
        ws_height = ws.height
        ws_width = ws.width
        ws_profile = ws.profile.copy()
        pixel_size = ws_transform.a
    
    river = gpd.read_file(
        r"D:\Ralph\research\common_maps\1-3-5_PH_River_MO-GED_2020.shp"
    )
    if river.crs != ws_crs:
        river = river.to_crs(ws_crs)

    river_raster = features.rasterize(
        [(geom, 1) for geom in river.geometry],
        out_shape=(ws_height, ws_width),
        transform=ws_transform,
        fill=0,
        all_touched=True,
        dtype=np.uint8
    )

    # Save normalized raster
    river_raster_path = r"Base_Layers\Upper_Marikina_WS_River_rasterized.tif"
    with rasterio.open(river_raster_path, "w", **ws_profile) as dst:
        dst.write(river_raster.astype("float32"), 1)

    print(f"Saved rasterized {river_raster_path}")


    clipped_river = np.where(ws_data == 1, river_raster == 1, 0)
        
    out_path = r"Base_Layers\Upper_Marikina_River_Clipped.tif"

    with rasterio.open(
        out_path,
        "w",
        driver="GTiff",
        height=ws_height,
        width=ws_width,
        count=1,
        dtype=np.uint8,
        crs=ws_crs,
        transform=ws_transform,
        compress="lzw"
    ) as dst:
        dst.write(clipped_river, 1)

    # -------------------------
    # Compute Euclidean distance (in pixels) then convert to meters
    # -------------------------
    inverted = 1 - clipped_river      # river=0, others=1
    distance_px = distance_transform_edt(inverted)
    distance_meters = distance_px * pixel_size

    # Ensure river cells = 0
    distance_meters[river_raster == 1] = 0

    # -------------------------
    # Clip outside AOI **before normalization**
    # -------------------------
    distance_meters = np.where(ws_data == 1, distance_meters, np.nan)

    # -------------------------
    # Save raw distance raster
    # -------------------------
    raw_path = r"Component_Layers\distance_to_river.tif"
    with rasterio.open(raw_path, "w", **ws_profile) as dst:
        dst.write(distance_meters.astype("float32"), 1)
    print(f"Saved raw distance raster: {raw_path}")

    # -------------------------
    # Normalize (0–1)
    # -------------------------
    max_val = np.nanmax(distance_meters)
    min_nonzero = np.nanmin(distance_meters[(distance_meters > 0)])

    distance_norm = 1 - ((distance_meters - min_nonzero) / (max_val - min_nonzero))
    distance_norm[river_raster == 1] = 0   # river always 0

    # -------------------------
    # Clip outside AOI **before normalization**
    # -------------------------
    distance_norm = np.where(ws_data == 1, distance_norm, np.nan)

    # Save normalized raster
    norm_path = r"Component_Layers\distance_to_river_normalized.tif"
    with rasterio.open(norm_path, "w", **ws_profile) as dst:
        dst.write(distance_norm.astype("float32"), 1)

    print(f"Saved normalized river distance raster: {norm_path}")

    return distance_norm

def fuzzy_river(aoi_mask, aoi_profile):
    raster_path = r"Component_Layers\distance_to_river_normalized.tif"
    layer = "river"
    print(f"Computing fuzzy membership for {layer}…")

    with rasterio.open(raster_path) as src:
        raster = src.read(1).astype(float)  # already aligned
        profile = src.profile

    raster_min = np.nanmin(raster)
    raster_max = np.nanmax(raster)
    print(f"River distance min = {raster_min}, max = {raster_max}")

    # Rescale explicitly 0–1 (in case any rounding)
    denom = (raster_max - raster_min)
    raster_norm = np.zeros_like(raster) if denom == 0 else (raster - raster_min) / denom

    # Fuzzy membership functions
    low_river      = trapezoid(raster_norm, 0.0, 0.0, 0.2, 1)
    moderate_river = trapezoid(raster_norm, 0.0, 0.4, 0.6, 1)
    high_river     = trapezoid(raster_norm, 0.0, 0.8, 1.0, 1.0)

    # Save outputs — all using AOI grid profile
    for name, data in zip(["low", "moderate", "high"],
                          [low_river, moderate_river, high_river]):

        out_path = rf"Fuzzy_Layers\river_fuzzy_{name}.tif"
        with rasterio.open(out_path, "w", **aoi_profile) as dst:
            dst.write(data.astype("float32"), 1)
        print(f"Saved fuzzy layer: {out_path}")

    print("Fuzzy river layers complete.")
    return low_river, moderate_river, high_river, aoi_profile


SLOPE

QGIS version: 3.40.2-Bratislava
QGIS code revision: 14826ca1
Qt version: 5.15.13
Python version: 3.12.8
GDAL version: 3.9.3
GEOS version: 3.13.0-CAPI-1.19.0
PROJ version: Rel. 9.5.0, September 15th, 2024
PDAL version: 2.8.1 (git-version: a06325)
Algorithm started at: 2025-12-18T15:27:59
Algorithm 'Slope' starting…
Input parameters:
{ 'AS_PERCENT' : False, 'BAND' : 1, 'COMPUTE_EDGES' : False, 'EXTRA' : '', 'INPUT' : 'D:/Ralph/research/common_maps/DEM_ifsar_30m_phil.tif', 'OPTIONS' : 'COMPRESS=DEFLATE|PREDICTOR=2|ZLEVEL=9', 'OUTPUT' : 'D:/Ralph/research/common_maps/slope_from_DEM_ifsar_30m.tif', 'SCALE' : 1, 'ZEVENBERGEN' : False }

GDAL command:
gdaldem slope D:/Ralph/research/common_maps/DEM_ifsar_30m_phil.tif D:/Ralph/research/common_maps/slope_from_DEM_ifsar_30m.tif -of GTiff -b 1 -s 1.0 -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=9
GDAL command output:

ASPECT

QGIS version: 3.40.2-Bratislava
QGIS code revision: 14826ca1
Qt version: 5.15.13
Python version: 3.12.8
GDAL version: 3.9.3
GEOS version: 3.13.0-CAPI-1.19.0
PROJ version: Rel. 9.5.0, September 15th, 2024
PDAL version: 2.8.1 (git-version: a06325)
Algorithm started at: 2025-12-18T15:35:37
Algorithm 'Aspect' starting…
Input parameters:
{ 'BAND' : 1, 'COMPUTE_EDGES' : False, 'EXTRA' : '', 'INPUT' : 'D:/Ralph/research/common_maps/DEM_ifsar_30m_phil.tif', 'OPTIONS' : 'COMPRESS=DEFLATE|PREDICTOR=2|ZLEVEL=9', 'OUTPUT' : 'D:/Ralph/research/common_maps/aspect_from_DEM_ifsar_30m.tif', 'TRIG_ANGLE' : False, 'ZERO_FLAT' : False, 'ZEVENBERGEN' : False }

GDAL command:
gdaldem aspect D:/Ralph/research/common_maps/DEM_ifsar_30m_phil.tif D:/Ralph/research/common_maps/aspect_from_DEM_ifsar_30m.tif -of GTiff -b 1 -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=9
GDAL command output:
0...10...20...30...40...50...60...70...80...90...100 - done.
Process completed successfully
Execution completed in 117.02 seconds (1 minute 57 seconds)
Results:
  OUTPUT: D:/Ralph/research/common_maps/aspect_from_DEM_ifsar_30m.tif

Loading resulting layers
Algorithm 'Aspect' finished


Fuzzy Gamma Operator

Fuzzy GAMMA combination (final fusion across all layers)

This is considered the “best-practice” operator in fuzzy GIS, recommended by:

ESRI ModelBuilder
IDRISI/TerrSet
Fuzzy MCE literature (Zadeh, Bonham-Carter, Eastman)


In [ ]:
import numpy as np
import rasterio
import os
# Example: two fuzzy membership rasters
# elev_high = ... (numpy array 0-1)
# rain_high = ... (numpy array 0-1)
# USE FUZZY GAMMA OPERATOR

def final_suitability(inputs,profile):

    gamma = 0.5  # fuzzy gamma parameter

    fuzzy_gamma_outputs = {}

    for cls, array in inputs.items():

        elev_fuzzy  = array["elev"]
        AR_fuzzy    = array["AR"]
        river_fuzzy = array["river"]

        # Fuzzy AND (algebraic product)
        fuzzy_and = elev_fuzzy * AR_fuzzy * river_fuzzy

        # Fuzzy OR (algebraic sum)
        values = [elev_fuzzy, AR_fuzzy, river_fuzzy]  # any number of components
        fuzzy_or = 1 - np.prod([1 - v for v in values])
        
        # Fuzzy Gamma
        suit_fuzzy = (fuzzy_and ** (1 - gamma)) * (fuzzy_or ** gamma)

        # clip 0–1 for safety
        suit_fuzzy = np.clip(suit_fuzzy, 0, 1)

        # store result
        fuzzy_gamma_outputs[cls] = suit_fuzzy


        out_path = rf"Fuzzy_Layers\gamma_{cls}.tif"
        profile.update(dtype=rasterio.float32, count=1)
        with rasterio.open(out_path, "w", **profile) as dst:
            dst.write(suit_fuzzy.astype(rasterio.float32), 1)
            
        print(f"Computed fuzzy gamma suitability for class: {cls}")


    # fuzzy_gamma_outputs = {"low": low_gamma, "mod": mod_gamma, "high": high_gamma}

    # Extract rasters
    G_low  = fuzzy_gamma_outputs["low"]
    G_mod  = fuzzy_gamma_outputs["mod"]
    G_high = fuzzy_gamma_outputs["high"]

    layers = [G_low, G_mod, G_high]

    # ---------------------------------------------------
    # 1) Fuzzy AND across classes (algebraic product)
    # ---------------------------------------------------
    fuzzy_and = np.ones_like(G_low, dtype=float)
    for arr in layers:
        fuzzy_and *= arr   # multiply per pixel

    # ---------------------------------------------------
    # 2) Fuzzy OR across classes (algebraic sum)
    # ---------------------------------------------------
    prod_complements = np.ones_like(G_low, dtype=float)
    for arr in layers:
        prod_complements *= (1 - arr)

    fuzzy_or = 1 - prod_complements

    # ---------------------------------------------------
    # 3) Fuzzy GAMMA operator across classes
    # ---------------------------------------------------

    out_path = rf"Fuzzy_Layers\final_suitability_fuzzyAND.tif"
    profile.update(dtype=rasterio.float32, count=1)

    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(fuzzy_and.astype(rasterio.float32), 1)

    print(f"Final suitability computed and saved using fuzzyAND.")

    out_path = rf"Fuzzy_Layers\final_suitability_fuzzyOR.tif"
    profile.update(dtype=rasterio.float32, count=1)

    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(fuzzy_or.astype(rasterio.float32), 1)

    print(f"Final computed and saved using fuzzyOR.")

    gamma = 0  # final fuzzy gamma parameter (choose 0 more restrictive, 1 more liberal)

    for i in range(1, 10):
        gamma = i / 10
        final_suitability = (fuzzy_and ** (1 - gamma)) * (fuzzy_or ** gamma)
        final_suitability = np.clip(final_suitability, 0, 1)

        # ---------------------------------------------------
        # Save output
        # ---------------------------------------------------
        out_path = rf"Fuzzy_Layers\final_suitability_fuzzygamma_{gamma}.tif"
        profile.update(dtype=rasterio.float32, count=1)

        with rasterio.open(out_path, "w", **profile) as dst:
            dst.write(final_suitability.astype(rasterio.float32), 1)

        print(f"Final suitability computed and saved using fuzzyGAMMA = {gamma}.")


In [ ]:
import numpy as np
import rasterio
import os

npver = np.__version__
rasteriover = rasterio.__version__

print(f"Numpy Version = {npver}") #Numpy Version = 2.3.5
print(f"Rasterio Version = {rasteriover}") #Rasterio Version = 1.4.3

shapefile_boundary = r"Base_Layers\Upper_Marikina_Watershed.shp"
raster_boundary = r"Base_Layers\Upper_Marikina_Watershed_rasterized.tif"
#aoi_mask, aoi_profile = generate_aoi_mask(shapefile_boundary)
#component_rasters = clip_component_layers(raster_boundary)

#low_AR, mod_AR, high_AR, profile = fuzzy_AR(aoi_mask, aoi_profile)
#low_elev, mod_elev, high_elev, profile = fuzzy_Elev(aoi_mask, aoi_profile)
#river_distance = river_distance_raster()
#low_river, mod_river, high_river, profile = fuzzy_river(aoi_mask, aoi_profile)

# fuzzy_inputs = {
#     "low": {
#         "elev": low_elev,
#         "AR": low_AR,
#         "river": low_river
#     },
#     "mod": {
#         "elev": mod_elev,
#         "AR": mod_AR,
#         "river": mod_river
#     },
#     "high": {
#         "elev": high_elev,
#         "AR": high_AR,
#         "river": high_river
#     }
# }

# final_suitability = final_suitability(fuzzy_inputs,profile)

Numpy Version = 2.3.5
Rasterio Version = 1.4.3
